In [1]:
import os
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, recall_score
from sklearn.cluster import KMeans, MiniBatchKMeans
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import scipy.io as sio
from tqdm import tqdm
import copy
import math
import random

# reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------- Load Data --------------------
data_path = r"C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\Dataset"
data = sio.loadmat(os.path.join(data_path, 'Pavia.mat'))['pavia']
labels = sio.loadmat(os.path.join(data_path, 'Pavia_gt.mat'))['pavia_gt']

# -------------------- Helper Functions --------------------
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=testRatio, random_state=randomState, stratify=y
    )
    return X_train, X_test, y_train, y_test

def applyPCA(X, numComponents=75):
    newX = X.reshape(-1, X.shape[2])
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = newX.reshape(X.shape[0], X.shape[1], numComponents)
    return newX, pca

def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0]+2*margin, X.shape[1]+2*margin, X.shape[2]))
    newX[margin:X.shape[0]+margin, margin:X.shape[1]+margin, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels=True):
    margin = (windowSize - 1) // 2
    paddedX = padWithZeros(X, margin=margin)
    patchesData = np.zeros((X.shape[0]*X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0]*X.shape[1]))
    idx = 0
    for r in range(margin, paddedX.shape[0]-margin):
        for c in range(margin, paddedX.shape[1]-margin):
            patch = paddedX[r-margin:r+margin+1, c-margin:c+margin+1]
            patchesData[idx] = patch
            patchesLabels[idx] = y[r-margin, c-margin]
            idx += 1
    if removeZeroLabels:
        mask = patchesLabels > 0
        patchesData = patchesData[mask]
        patchesLabels = patchesLabels[mask] - 1
    return patchesData, patchesLabels

# Dataset / preprocessing parameters (kept same as your code)
windowSize = 3
test_ratio = 0.25
dataset = 'IP'
K = 30 if dataset == 'IP' else 15

X, pca = applyPCA(data, numComponents=K)
X, y = createImageCubes(X, labels, windowSize=windowSize)

# Split and convert to torch tensors
X_train_np, X_test_np, y_train_np, y_test_np = splitTrainTestSet(X, y, test_ratio)
X_train = torch.tensor(X_train_np, dtype=torch.float32, device=DEVICE)
y_train = torch.tensor(y_train_np, dtype=torch.long, device=DEVICE)
X_test  = torch.tensor(X_test_np, dtype=torch.float32, device=DEVICE)
y_test  = torch.tensor(y_test_np, dtype=torch.long, device=DEVICE)

S = X_train.shape[1]
L = X_train.shape[3]
n_classes = len(np.unique(y_train_np))

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)

X_train shape: torch.Size([111114, 3, 3, 30])
X_test  shape: torch.Size([37038, 3, 3, 30])


In [2]:
class SpectralSpatialTransformer(nn.Module):
    """
    Spectral-Spatial Transformer head replacing the conv2d + flatten step.
    Keeps the 3D conv stem similar to HybridSN and then applies a Transformer
    over spatial tokens where each token is a (channel * depth) feature vector.
    """
    def __init__(self, input_shape, n_classes,
                 embed_dim=256, n_heads=8, n_layers=4, ff_dim=512, attn_dropout=0.0, drop=0.4):
        super(SpectralSpatialTransformer, self).__init__()
        # input_shape expected as (S, _, L) or (S, S, L) - we only need S and L
        S, _, L = input_shape

        # 3D conv stem (keeps same kernel choices as your HybridSN)
        self.conv3d_1 = nn.Conv3d(1, 8, kernel_size=(3,3,7))
        self.conv3d_2 = nn.Conv3d(8,16, kernel_size=(1,1,5))
        self.conv3d_3 = nn.Conv3d(16,32, kernel_size=(1,1,3))

        # Transformer projection + encoder
        self.embed_dim = embed_dim
        # token projection (project per-token feature vector to embed_dim)
        self.token_proj = None  # will be created after dummy forward
        self.pos_emb = None
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=n_heads,
            dim_feedforward=ff_dim,
            dropout=attn_dropout,
            batch_first=True  # so inputs are (B, seq_len, embed_dim)
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # FC head (kept same sizes as your original code)
        self.fc1 = None
        self.dropout1 = nn.Dropout(drop)
        self.fc2 = None
        self.dropout2 = nn.Dropout(drop)
        self.fc3 = None

        # Build modules that depend on S/L by running a dummy forward pass
        with torch.no_grad():
            # create dummy input in same layout as forward: (B,1,H,W,L)
            x_dummy = torch.zeros(1,1,S,S,L)
            x_dummy = F.relu(self.conv3d_1(x_dummy))
            x_dummy = F.relu(self.conv3d_2(x_dummy))
            x_dummy = F.relu(self.conv3d_3(x_dummy))
            B, C, H, W, D = x_dummy.shape  # H,W are spatial dims after convs
            token_dim = C * D              # per-spatial-location vector size

            # token projection: project token_dim -> embed_dim
            self.token_proj = nn.Linear(token_dim, embed_dim)

            # positional embedding for H*W tokens (learnable)
            seq_len = H * W
            self.pos_emb = nn.Parameter(torch.zeros(1, seq_len, embed_dim))

            # transformer's output is pooled -> pass to FCs
            # we'll pool to get a fixed vector of size embed_dim
            self.fc1 = nn.Linear(embed_dim, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, n_classes)

            # initialize pos_emb small
            nn.init.trunc_normal_(self.pos_emb, std=0.02)
            # init token_proj
            nn.init.xavier_uniform_(self.token_proj.weight)
            if self.token_proj.bias is not None:
                nn.init.zeros_(self.token_proj.bias)

    def forward(self, x):
        # x : (B, H, W, L) as in your dataset tensors
        x = x.unsqueeze(1)  # -> (B,1,H,W,L)
        x = F.relu(self.conv3d_1(x))
        x = F.relu(self.conv3d_2(x))
        x = F.relu(self.conv3d_3(x))
        B, C, H, W, D = x.shape

        # reshape to tokens: (B, H*W, C*D)
        x = x.view(B, C*D, H, W)          # (B, C*D, H, W)
        x = x.permute(0, 2, 3, 1).contiguous()  # (B, H, W, C*D)
        x = x.view(B, H*W, C*D)           # (B, seq_len, token_dim)

        # project tokens to embedding dim
        x = self.token_proj(x)            # (B, seq_len, embed_dim)

        # add positional embedding (broadcast batch)
        # self.pos_emb shape: (1, seq_len, embed_dim)
        x = x + self.pos_emb

        # transformer encoding (batch_first=True)
        x = self.transformer(x)           # (B, seq_len, embed_dim)

        # global pooling (mean over tokens)
        x = x.mean(dim=1)                 # (B, embed_dim)

        # fc head (similar to HybridSN)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)                   # (B, n_classes)
        return x


In [3]:

# -------------------- Training --------------------
def train_model(model, X_train, y_train, epochs=1, lr=0.001, batch_size=None):
    # Note: original code did full-batch; keep same behavior for fairness unless batch_size is given
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    t0 = time.time()
    
    if batch_size is None:
        # full-batch (original)
        for epoch in tqdm(range(epochs), desc="Training", ncols=100, unit="epoch"):
            optimizer.zero_grad()
            outputs = model(X_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer.step()
            tqdm.write(f"Epoch [{epoch+1}/{epochs}] - Loss: {loss.item():.4f}")
    else:
        # mini-batch training
        dataset = torch.utils.data.TensorDataset(X_train, y_train)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        for epoch in tqdm(range(epochs), desc="Training", ncols=100, unit="epoch"):
            epoch_loss = 0.0
            for xb, yb in loader:
                optimizer.zero_grad()
                outputs = model(xb)
                loss = criterion(outputs, yb)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            tqdm.write(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss/len(loader):.4f}")
    
    total_time = time.time() - t0
    return model, total_time

# -------------------- Evaluation --------------------
def evaluate_model(model, X_test, y_test):
    model.eval()
    t1 = time.time()
    with torch.no_grad():
        outputs = model(X_test)
        preds = outputs.argmax(dim=1).cpu().numpy()
    y_true = y_test.cpu().numpy()
    acc = np.mean(preds==y_true)
    f1 = f1_score(y_true, preds, average='macro')
    recall = recall_score(y_true, preds, average='macro')
    inf_time = time.time() - t1
    return inf_time, acc, f1, recall

# -------------------- DKM (existing) --------------------
def dkm_cluster_weights(model, k=8, iterations=10, layer_mask=None):
    """
    layer_mask: function name->bool to indicate whether to cluster that parameter. If None cluster all.
    Returns: compressed_params_effective (estimate in terms of k * vector_dim per clustered weight)
    """
    compressed_params = 0
    for name, param in model.named_parameters():
        if "weight" in name:
            if (layer_mask is not None) and (not layer_mask(name)):
                # skip clustering for this weight
                compressed_params += param.numel()
                continue
            W = param.data
            W_flat = W.view(W.shape[0], -1).clone()  # (out_channels, vector_dim)
            # initialize centroids by random selection
            idx = torch.randperm(W_flat.size(0))[:k]
            C = W_flat[idx].clone()
            for _ in range(iterations):
                dist = torch.cdist(W_flat, C)  # (N, k)
                A = torch.softmax(-dist, dim=1)  # soft assignments across k
                C = torch.mm(A.t(), W_flat) / (A.sum(0).unsqueeze(1)+1e-6)
            clustered_W = torch.mm(A, C)
            param.data = clustered_W.view(param.shape)
            compressed_params += k * W_flat.shape[1]  # store effective param count for cluster codebook
        elif "bias" in name:
            compressed_params += param.numel()
    return compressed_params

# -------------------- KMeans / MiniBatchKMeans clustering adapted to weights --------------------
def kmeans_cluster_weights(model, k=8, iterations=20, layer_mask=None, minibatch=False):
    compressed_params = 0
    for name, param in model.named_parameters():
        if "weight" in name:
            if (layer_mask is not None) and (not layer_mask(name)):
                compressed_params += param.numel()
                continue
            W = param.data
            W_flat = W.view(W.shape[0], -1).cpu().numpy()  # shape (N, D)
            if W_flat.shape[0] <= k:
                # cannot cluster fewer vectors than k
                compressed_params += W_flat.size
                continue
            if minibatch:
                km = MiniBatchKMeans(n_clusters=k, random_state=SEED, max_iter=iterations)
            else:
                km = KMeans(n_clusters=k, random_state=SEED, n_init=3, max_iter=iterations)
            labels = km.fit_predict(W_flat)
            centroids = km.cluster_centers_  # (k, D)
            # assign back using centroid of assigned cluster (hard assignment)
            clustered = centroids[labels]
            clustered_t = torch.tensor(clustered, dtype=param.dtype, device=param.device)
            param.data = clustered_t.view(param.shape)
            compressed_params += k * W_flat.shape[1]
        elif "bias" in name:
            compressed_params += param.numel()
    return compressed_params

# -------------------- EFDPC (density peak clustering variant) for weights --------------------
def efdpc_cluster_weights(model, k=8, layer_mask=None):
    """
    Adaptation of EFDPC (density-peak based) for clustering weight-vectors.
    We compute pairwise distances between weight vectors in a layer,
    compute density (rho) using exponential kernel, compute delta (distance to nearest higher-rho),
    compute gamma = rho * delta^2, pick top-k as centroids, and soft-assign using softmax(-dist).
    Returns compressed params estimate k * vector_dim per clustered weight.
    """
    compressed_params = 0
    for name, param in model.named_parameters():
        if "weight" in name:
            if (layer_mask is not None) and (not layer_mask(name)):
                compressed_params += param.numel()
                continue
            W = param.data
            W_flat = W.view(W.shape[0], -1).cpu().numpy()  # (N, D)
            N = W_flat.shape[0]
            if N <= k:
                compressed_params += W_flat.size
                continue
            # compute pairwise distance matrix (NxN)
            # to save memory, compute condensed distances
            from scipy.spatial.distance import pdist, squareform
            dist_vec = pdist(W_flat, metric='euclidean')
            Dist_matrix = squareform(dist_vec)
            # choose dc using a small percentile, similar to MATLAB percent=2
            percent = 2.0
            tril_vals = Dist_matrix[np.tril_indices(N, k=-1)]
            sda = np.sort(tril_vals)
            if len(sda) == 0:
                dc = 1.0
            else:
                pos = max(0, int(len(sda) * (percent/100.0) ) - 1)
                pos = min(len(sda)-1, pos)
                dini = sda[pos]
                # add small epsilon to avoid zero
                dc = max(dini / math.exp(10.0/N), 1e-6)

            # compute rho
            rho = np.zeros(N)
            for i in range(N):
                # exponential kernel
                rho[i] = np.sum(np.exp(- (Dist_matrix[i,:] / (dc+1e-12))**2 )) - 1.0  # exclude self
            # compute delta
            ordrho = np.argsort(-rho)  # descending by rho
            delta = np.zeros(N)
            maxD = np.max(Dist_matrix)
            delta[ordrho[0]] = maxD
            for idx in range(1, N):
                i = ordrho[idx]
                # distance to nearest higher density point
                higher = ordrho[:idx]
                delta[i] = np.min(Dist_matrix[i, higher])
            # normalize rho and delta
            if rho.max() - rho.min() > 0:
                rho_n = (rho - rho.min()) / (rho.max() - rho.min())
            else:
                rho_n = rho
            if delta.max() - delta.min() > 0:
                delta_n = (delta - delta.min()) / (delta.max() - delta.min())
            else:
                delta_n = delta
            gamma = rho_n * (delta_n**2)
            # pick top-k gamma indices as centroids
            topk_idx = np.argsort(-gamma)[:k]
            C = W_flat[topk_idx]  # (k, D)
            # soft assignment via distances
            from scipy.spatial.distance import cdist
            dist = cdist(W_flat, C)  # (N, k)
            A = np.exp(-dist)  # soft weights
            A = A / (A.sum(axis=1, keepdims=True) + 1e-12)
            clustered = A.dot(C)
            clustered_t = torch.tensor(clustered, dtype=param.dtype, device=param.device)
            param.data = clustered_t.view(param.shape)
            compressed_params += k * W_flat.shape[1]
        elif "bias" in name:
            compressed_params += param.numel()
    return compressed_params

# -------------------- Layer selection helper --------------------
def make_layer_mask_exclude_first_last(model):
    """
    Returns a function that given parameter name returns True if that parameter should be clustered.
    This implementation finds the ordered list of 'weight' parameter names and excludes the first and last.
    """
    weight_names = [name for name, _ in model.named_parameters() if "weight" in name]
    exclude = set()
    if len(weight_names) >= 1:
        exclude.add(weight_names[0])
    if len(weight_names) >= 2:
        exclude.add(weight_names[-1])
    def mask(name):
        if "weight" not in name:
            return False  # don't cluster biases in layer-selection sense (we keep biases unclustered count-wise above)
        return name not in exclude
    return mask


In [4]:

def make_layer_mask_all(model):
    def mask(name):
        return "weight" in name
    return mask

# -------------------- Utility: count total params --------------------
def count_params(model):
    return sum(p.numel() for p in model.parameters())

# -------------------- Utility: get compressed params from clustering run (already returned) --------------------

# -------------------- Utility: estimate inference time proportionally --------------------
def estimate_inf_time(num_params, baseline_params, baseline_inf_time):
    # avoid zero division
    if baseline_params == 0:
        return baseline_inf_time
    return baseline_inf_time * (num_params / baseline_params)

# -------------------- Run Baseline (kept exactly like your original configuration 1) --------------------
model = SpectralSpatialTransformer((S,S,L), n_classes).to(DEVICE)
print("\n-- Training baseline model (this preserves your original configuration 1 behavior) --")
model, train_time = train_model(model, X_train, y_train, epochs=100)  # identical to your original
inf_time_measured, acc, f1, rec = evaluate_model(model, X_test, y_test)

baseline_params = count_params(model)
baseline_inf_time_measured = inf_time_measured
baseline_results = {
    "train_time": train_time,
    "measured_inf_time": baseline_inf_time_measured,
    "params": baseline_params,
    "acc": acc,
    "f1": f1,
    "recall": rec
}

print("\n===== Baseline HybridSN =====")
print(f"Training Time : {train_time:.3f} sec")
print(f"Measured Inference Time: {inf_time_measured:.6f} sec")
print(f"Parameters    : {baseline_params}")
print(f"Accuracy      : {acc:.4f}")
print(f"F1 Score      : {f1:.4f}")
print(f"Recall        : {rec:.4f}")

# -------------------- Full experimental grid --------------------
# Training stages: 'posthoc' (full baseline train -> cluster -> finetune),
#                  'partial' (warm-up epochs -> cluster -> finetune)
training_stages = ['posthoc', 'partial']
layer_selections = ['all', 'exclude_first_last']
clustering_methods = ['KMeans', 'MiniBatchKMeans', 'EFDPC']  # 4 clustering options (DKM is original)

# hyperparameters for experiments
k_values = [4, 8, 16, 32]  # cluster counts to test
warmup_epochs = 50  # partial warm-up epochs (you can change)
fine_tune_epochs = 100  # fine-tune after clustering
lr_finetune = 0.001

# We'll store results
exp_results = []

# For fairness, reuse baseline trained model weights as starting point (posthoc) or for partial warmup start from scratch and warmup
for stage in training_stages:
    for layer_sel in layer_selections:
        for method in clustering_methods:
            for k in k_values:
                # Prepare a fresh model for this run
                model_k = SpectralSpatialTransformer((S,S,L), n_classes).to(DEVICE)
                # Copy baseline weights for posthoc scenario
                if stage == 'posthoc':
                    model_k.load_state_dict(model.state_dict())
                    warmup_time = 0.0
                elif stage == 'partial':
                    # For partial warmup we start from new random init and train warmup_epochs
                    model_k, warmup_time = train_model(model_k, X_train, y_train, epochs=warmup_epochs, lr=0.001)
                else:
                    raise ValueError("Unknown training stage")

                # Choose layer mask
                if layer_sel == 'all':
                    layer_mask = make_layer_mask_all(model_k)
                else:
                    layer_mask = make_layer_mask_exclude_first_last(model_k)

                # Apply clustering based on method
                if method == 'DKM':
                    compressed_params = dkm_cluster_weights(model_k, k=k, iterations=10, layer_mask=layer_mask)
                elif method == 'KMeans':
                    compressed_params = kmeans_cluster_weights(model_k, k=k, iterations=20, layer_mask=layer_mask, minibatch=False)
                elif method == 'MiniBatchKMeans':
                    compressed_params = kmeans_cluster_weights(model_k, k=k, iterations=50, layer_mask=layer_mask, minibatch=True)
                elif method == 'EFDPC':
                    compressed_params = efdpc_cluster_weights(model_k, k=k, layer_mask=layer_mask)
                else:
                    raise ValueError("Unknown clustering method")

                # Fine-tune the clustered model
                model_k, ft_time = train_model(model_k, X_train, y_train, epochs=fine_tune_epochs, lr=lr_finetune)

                # Evaluate after clustering + fine-tuning
                inf_time_measured_k, acc_k, f1_k, rec_k = evaluate_model(model_k, X_test, y_test)
                total_params_k = count_params(model_k)

                # Compute estimated inference time based only on parameter count relative to baseline
                estimated_inf_time_k = estimate_inf_time(total_params_k, baseline_params, baseline_inf_time_measured)

                # Save & print results
                result = {
                    "stage": stage,
                    "layer_selection": layer_sel,
                    "method": method,
                    "k": k,
                    "warmup_time": warmup_time if stage == 'partial' else 0.0,
                    "cluster_time": None,
                    "fine_tune_time": ft_time,
                    "measured_inf_time": inf_time_measured_k,
                    "estimated_inf_time": estimated_inf_time_k,
                    "params": total_params_k,
                    "compressed_params_effective": compressed_params,
                    "acc": acc_k,
                    "f1": f1_k,
                    "recall": rec_k
                }
                exp_results.append(result)

                # Print concise summary for this config
                print("\n----- Experiment Result -----")
                print(f"Stage           : {stage}")
                print(f"Layer selection : {layer_sel}")
                print(f"Clustering      : {method}")
                print(f"k (clusters)    : {k}")
                if stage == 'partial':
                    print(f"Warmup Time     : {warmup_time:.3f} sec")
                print(f"Fine-tune Time  : {ft_time:.3f} sec")
                print(f"Measured Inference Time: {inf_time_measured_k:.6f} sec")
                print(f"Estimated Inference Time (proportional to params): {estimated_inf_time_k:.6f} sec")
                print(f"Parameters (model): {total_params_k}")
                print(f"Compressed Params (effective/codebook): {compressed_params}")
                print(f"Accuracy         : {acc_k:.4f}")
                print(f"F1 Score         : {f1_k:.4f}")
                print(f"Recall           : {rec_k:.4f}")

# Optionally, print a short table summary
print("\n\n===== Summary of all experiments (first 10 shown) =====")
for r in exp_results[:10]:
    print(f"{r['stage']}/{r['layer_selection']}/{r['method']}/k={r['k']} -> params: {r['params']}, est_inf_time: {r['estimated_inf_time']:.6f}, acc: {r['acc']:.4f}, f1: {r['f1']:.4f}")

# Save results to a numpy file for downstream analysis if desired
import json
out_file = r"C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Pavia_SST.json"
with open(out_file, "w") as f:
    json.dump(exp_results, f, indent=2)
print(f"\nSaved experiment results to {out_file}")



-- Training baseline model (this preserves your original configuration 1 behavior) --


Training:   1%|▌                                                 | 1/100 [00:26<43:33, 26.40s/epoch]

Epoch [1/100] - Loss: 2.1934


Training:   2%|█                                                 | 2/100 [00:56<46:57, 28.75s/epoch]

Epoch [2/100] - Loss: 1.9065


...

Epoch [98/100] - Loss: 0.2352


Training:  99%|████████████████████████████████████████████████▌| 99/100 [39:33<00:23, 23.53s/epoch]

Epoch [99/100] - Loss: 0.2297


Training: 100%|████████████████████████████████████████████████| 100/100 [39:57<00:00, 23.97s/epoch]


Epoch [100/100] - Loss: 0.2229

----- Experiment Result -----
Stage           : partial
Layer selection : exclude_first_last
Clustering      : EFDPC
k (clusters)    : 32
Warmup Time     : 1193.366 sec
Fine-tune Time  : 2397.476 sec
Measured Inference Time: 2.380278 sec
Estimated Inference Time (proportional to params): 2.867512 sec
Parameters (model): 2358969
Compressed Params (effective/codebook): 212665
Accuracy         : 0.9360
F1 Score         : 0.6037
Recall           : 0.6551


===== Summary of all experiments (first 10 shown) =====
posthoc/all/KMeans/k=4 -> params: 2358969, est_inf_time: 2.867512, acc: 0.9627, f1: 0.8156
posthoc/all/KMeans/k=8 -> params: 2358969, est_inf_time: 2.867512, acc: 0.9900, f1: 0.9645
posthoc/all/KMeans/k=16 -> params: 2358969, est_inf_time: 2.867512, acc: 0.9945, f1: 0.9826
posthoc/all/KMeans/k=32 -> params: 2358969, est_inf_time: 2.867512, acc: 0.9950, f1: 0.9844
posthoc/all/MiniBatchKMeans/k=4 -> params: 2358969, est_inf_time: 2.867512, acc: 0.9584, 